## Libraries

In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install sentencepiece
  !pip install wandb
  !pip install faiss-cpu
  !pip install bitsandbytes
  !pip install accelerate
  !pip install sentencepiece
  !pip install protobuf
  !pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["WANDB_DISABLED"] = "true"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["TF_USE_LEGACY_KERAS"] = "1"
os.environ["HF_HUB_DISABLE_XET"] = "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import torch

if IN_COLAB:
    root_path = '/content/drive/My Drive/instructABSA'
else:
    root_path = 'Enter local path'

use_mps = True if torch.backends.mps.is_built() else False
os.chdir(root_path)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5GeneratorLLMOps, T5Classifier
from instructions import InstructionsHandler

## Training

In [ ]:
task_name = 'aoste-4'
experiment_name = 'aoste-flan-t5-base'
model_checkpoint = 'google/flan-t5-base'
print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '-')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  aoste-flan-t5-base
Model output path:  ./Models/aoste-4/google-flan-t5-base-aoste-flan-t5-base


In [ ]:
# Load the data
id_train_file_path = './Data/train_aoste.json'  ### <-- Update with your data
id_test_file_path = './Data/test_aoste.json'   ### <-- Update with your data

id_tr_df = pd.read_json(id_train_file_path)
id_te_df = pd.read_json(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set2()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_aoste_format(
        df               = loader.train_df_id,
        key              = 'term',
        label_key        = 'polarity',
        text_col         = 'raw_words',
        aspect_col       = 'aspects',
        opinion_col      = 'opinions',
        bos_instruction  = instruct_handler.aoste['bos_instruct1'],
        eos_instruction  = instruct_handler.aoste['eos_instruct']
    )
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_aoste_format(
        df               = loader.test_df_id,
        key              = 'term',
        label_key        = 'polarity',
        text_col         = 'raw_words',
        aspect_col       = 'aspects',
        opinion_col      = 'opinions',
        bos_instruction  = instruct_handler.aoste['bos_instruct1'],
        eos_instruction  = instruct_handler.aoste['eos_instruct']
    )

In [ ]:
# Create T5 utils object
t5_exp = T5GeneratorLLMOps(
    model_checkpoint = model_checkpoint,
    max_new_tokens   = 128,
    use_lora         = True,                   # bật LoRA
    use_dora         = False,                  # nếu muốn thử DoRA thì set True
    lora_r           = 8,
    dora_r           = 4,
    quantize_fp16    = True,                   # load model 16‑bit
    cache_folder     = '/Models/faiss_cache',        # semantic cache
    wandb_project    = experiment_name         # project trong WandB
)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'eval_strategy': 'epoch',
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':16,
    'num_train_epochs':30,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lpphat22 (bigdata-project) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Map:   0%|          | 0/973 [00:00<?, ? examples/s]

Map:   0%|          | 0/243 [00:00<?, ? examples/s]

In [ ]:
id_tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['raw_words', 'words', 'aspects', 'opinions', 'labels', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 973
    })
    test: Dataset({
        features: ['raw_words', 'words', 'aspects', 'opinions', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 243
    })
})

In [ ]:
# Split the test dataset in half
train_test_split = id_tokenized_ds['test'].train_test_split(test_size=0.5)
train_test_split

DatasetDict({
    train: Dataset({
        features: ['raw_words', 'words', 'aspects', 'opinions', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 121
    })
    test: Dataset({
        features: ['raw_words', 'words', 'aspects', 'opinions', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 122
    })
})

In [ ]:
# Add the "new" 'train' and 'test' splits to the original DatasetDict with their new names
id_tokenized_ds['test'] = train_test_split['train']
id_tokenized_ds['validation'] = train_test_split['test']  # Use 'test' as the validation set
id_tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['raw_words', 'words', 'aspects', 'opinions', 'labels', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 973
    })
    test: Dataset({
        features: ['raw_words', 'words', 'aspects', 'opinions', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 121
    })
    validation: Dataset({
        features: ['raw_words', 'words', 'aspects', 'opinions', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 122
    })
})

In [ ]:
!python -m pip install --upgrade pip
!python -m pip install -U accelerate
!python -m pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.6.0
    Uninstalling accelerate-1.6.0:
      Successfully uninstalled accelerate-1.6.0


In [ ]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



Model training started ....


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,2.937500
2,No log,1.738281
3,No log,1.288086
4,No log,1.102539
5,2.184700,1.002930
6,2.184700,0.947266
7,2.184700,0.925781
8,2.184700,0.881348
9,1.065900,0.864258
10,1.065900,0.821289
